Làm sạch dữ liệu

In [1]:
import pandas as pd

restaurants_df = pd.read_csv("Get_Restaurant.csv")
items_df = pd.read_csv("Get_Items.csv")

In [6]:
# Xoá cột phụ và chuẩn hoá tên quán
restaurants_df_cleaned = restaurants_df.drop(columns=["Unnamed: 0", "Restaurant_Phone", "Restaurant_Phone.1"])
items_df_cleaned = items_df.drop(columns=["Unnamed: 0"])

# Chuẩn hoá tên quán để ghép bảng sau này
restaurants_df_cleaned["Restaurant_Name"] = restaurants_df_cleaned["Restaurant_Name"].str.strip()
items_df_cleaned["Restaurant_Name"] = items_df_cleaned["Restaurant_Name"].str.strip()


In [7]:
import re

# Hàm nâng cao trích quận từ địa chỉ
def extract_district(address):
    if pd.isna(address):
        return None
    # Tách từ sau "P." hoặc dấu phẩy
    tokens = address.split(",")
    for token in reversed(tokens):
        token = token.strip()
        if any(x in token.lower() for x in ["quận", "huyện", "tp", "thành phố", "thị xã", "cầu giấy", "đống đa"]):
            return token
    return tokens[-2].strip() if len(tokens) >= 2 else tokens[-1].strip()

# Áp dụng
restaurants_df_cleaned["District"] = restaurants_df_cleaned["Restaurant_Address"].apply(extract_district)


In [2]:
# Chuyển tất cả tên món về chữ thường để so sánh dễ hơn
items_df["Dish_Name_lower"] = items_df["Dish_Name"].str.lower()

# Danh sách từ khóa không phải món ăn (bạn có thể bổ sung thêm)
exclude_keywords = [
    "sofa", "ghế", "bàn", "quần", "áo", "váy", "giày", "tủ", "kệ",
    "nệm", "sách", "kem", "sữa rửa mặt", "sữa tắm", "bỉm", "xe đạp", 
    "đèn", "quạt", "máy", "móc khóa", "gối", "ga trải giường"
]

# Tạo biểu thức lọc (kết hợp OR các từ khóa)
pattern = '|'.join(exclude_keywords)

# Lọc ra các dòng KHÔNG chứa từ khóa loại trừ
items_df_cleaned_food_only = items_df[~items_df["Dish_Name_lower"].str.contains(pattern, na=False)]

# Xoá cột phụ nếu cần
items_df_cleaned_food_only = items_df_cleaned_food_only.drop(columns=["Dish_Name_lower"])


In [8]:
# Kiểm tra giá trị thiếu
print("Thiếu ở bảng quán ăn:")
print(restaurants_df_cleaned.isnull().sum())

print("\nThiếu ở bảng món ăn:")
print(items_df_cleaned.isnull().sum())


Thiếu ở bảng quán ăn:
Restaurant_Name       0
Restaurant_Address    0
Restaurant_Rate       0
Restaurant_Review     0
Restaurant_url        0
District              0
dtype: int64

Thiếu ở bảng món ăn:
Restaurant_Name    0
Menu_Name          0
Dish_Name          0
Dish_order         0
Dish_Price         0
Dish_Name_lower    0
dtype: int64


In [7]:
restaurants_df_cleaned.head(5)
items_df_cleaned.head(5)


,Restaurant_Name,Menu_Name,Dish_Name,Dish_order,Dish_Price
0,Circle K - Bia Lạnh Các Loại - 5-4A Trung Hòa ...,SNACK,Bánh Que Vị Tôm Dorkbua 55G (1 Gói)-8850157500371,10,19000.0
1,Circle K - Bia Lạnh Các Loại - 5-4A Trung Hòa ...,SNACK,Khoai Tây Chiên Vị Kim Chi O'Star 108G (1 Gói)...,10,25000.0
2,Circle K - Bia Lạnh Các Loại - 5-4A Trung Hòa ...,SNACK,Bánh Vị Cua Kinh Đô 32G (1 Gói)-8934680012652,10,7000.0
3,Circle K - Bia Lạnh Các Loại - 5-4A Trung Hòa ...,SNACK,Swing Khoai Tây Chiên Vị Bò BíT TếT 63G(1 ...,9,14000.0
4,Circle K - Bia Lạnh Các Loại - 5-4A Trung Hòa ...,SNACK,OISHI Snack Bắp Ngọt 73g (1 Gói)-8934803026078,8,13000.0


In [ ]:
# Lưu dữ liệu đã làm sạch thành file CSV
restaurants_df_cleaned.to_csv("restaurants_cleaned.csv", index=False)
items_df_cleaned.to_csv("items_cleaned.csv", index=False)


In [9]:
print("📦 Tổng số item gốc:", len(items_df))
print("🍽️ Tổng số item sau khi lọc:", len(items_df_cleaned))

# Xem 5 món đầu tiên trong file gốc
print("\n🔍 Một vài item trong file GỐC:")
print(items_df["Dish_Name"].head())

# Xem 5 món đầu tiên sau khi đã lọc sạch
print("\n✅ Một vài item trong file ĐÃ LỌC:")
print(items_df_cleaned["Dish_Name"].head())


📦 Tổng số item gốc: 533132
🍽️ Tổng số item sau khi lọc: 533132

🔍 Một vài item trong file GỐC:
0    Bánh Que Vị Tôm Dorkbua 55G (1 Gói)-8850157500371
1    Khoai Tây Chiên Vị Kim Chi O'Star 108G (1 Gói)...
2        Bánh Vị Cua Kinh Đô 32G (1 Gói)-8934680012652
3    Swing Khoai Tây Chiên Vị Bò BíT TếT 63G(1 ...
4    OISHI Snack Bắp Ngọt 73g (1 Gói)-8934803026078
Name: Dish_Name, dtype: object

✅ Một vài item trong file ĐÃ LỌC:
0    Bánh Que Vị Tôm Dorkbua 55G (1 Gói)-8850157500371
1    Khoai Tây Chiên Vị Kim Chi O'Star 108G (1 Gói)...
2        Bánh Vị Cua Kinh Đô 32G (1 Gói)-8934680012652
3    Swing Khoai Tây Chiên Vị Bò BíT TếT 63G(1 ...
4    OISHI Snack Bắp Ngọt 73g (1 Gói)-8934803026078
Name: Dish_Name, dtype: object


In [11]:
# Chuyển sang chữ thường để so sánh dễ hơn
items_df["Dish_Name_lower"] = items_df["Dish_Name"].str.lower()

# Tạo biểu thức từ khóa loại trừ
exclude_keywords = [
    "sofa", "ghế", "bàn", "quần", "áo", "váy", "giày", "tủ", "kệ",
    "nệm", "sách", "kem", "sữa rửa mặt", "sữa tắm", "bỉm", "xe đạp", 
    "đèn", "quạt", "máy", "móc khóa", "gối", "ga trải giường"
]
pattern = '|'.join(exclude_keywords)

# Lấy ra các dòng bị loại bỏ
excluded_items = items_df[items_df["Dish_Name_lower"].str.contains(pattern, na=False)]

print("🛑 Các sản phẩm đã bị loại bỏ (không phải món ăn):")
print(excluded_items[["Dish_Name", "Dish_Price"]].head())


🛑 Các sản phẩm đã bị loại bỏ (không phải món ăn):
                                              Dish_Name  Dish_Price
1315                Bàn Chải Massage Gắn Tường Cho Mèo      60000.0
1317  Bàn chải massage tạo bọt silicon( giao màu ngẫ...     40000.0
1332  Sữa Tắm Hantox Trị Bọ Chét, Ve, Ghẻ, Rận Màu X...     50000.0
1333                  Sữa Tắm Cho Chó SOS Mềm Mượt Lông    110000.0
1334                     Sữa Tắm Cho Chó Lông Trắng SOS    110000.0
1335                    Sữa Tắm SOS Cho Chó Lông Nâu Đỏ    110000.0
1336  Sữa Tắm Hantox Trị Bọ Chét, Ve, Ghẻ, Rận Màu H...     50000.0
1337               Sữa Tắm Cho Chó SOS Khử Mùi Tạo Kiểu    110000.0
1339               Sữa Tắm Cho Chó SOS Phục Hồi Da Lông    110000.0
1342                        Sữa Tắm Olive Cho Mèo 450ml     65000.0
